# In this code example I am exploring the distance 3 surface code
### The example here reported is a simplification to the case where the initial logic state is |0> or |+>, other types of states like i or -i requires topological state preparations, and fault tolerant protocols on their own, look at "twists"(https://arxiv.org/abs/2302.07395). Due to the nature fo the example, for each type of logic state it is possible to detect specific errors. On the |0>L, Z-flips are invisible, while on the |+>L X-flips are invisible. The circuit will be set on a specific initial state to play around with the detecton of one of the two errors. Code quality is this notebook has been neglected.

![example_of_surf](surface3dist.jpg)


In [1]:
import pennylane as qml


def surface_code_d3_circuit(is_plus, i_want_errors, line_of_error, corrections = []):
    """
    Implementation of circuit for dist 3 surface code.
    This type of surface code can detect only one physical error.
    For the sake of the example we need to choose a simple logic state, either |0> or |+>.
    
    Args:
    is_plus (bool): whether the initial state was |+> (True) or |0> (False)
    i_want_errors (bool): allows to inject a controlled error to verify that it works
    line_of_error (int): index of the line on which we want an error to appear
    
    Returns:
    list: list of 8 measurement results [X1,X2,X3,X4, Z1,Z3,Z2,Z4] !!!! Attention
    """

    # Data qubits: 0-8
    # Measurement qubits: 9-16 (9,12 for X-stabilizers, 13,16 for Z-stabilizers)
    
    # initial state is simply |+> on all the qubits
    if is_plus:
        for i in range(9):
            qml.Hadamard(wires=i)
        # apply a Z-flip
        if i_want_errors:
            print("Applied error Z")
            qml.Z(wires=line_of_error)
    else:
        # apply an X-flip
        if i_want_errors:
            print("Applied error X")
            qml.X(wires=line_of_error)
            
    # Applies error correction
    # ATTENTION : This is only for the sake of clarity, 
    # in the experiment correction is not applied but only tracked by software!!!
    for qubit, op in corrections:
        if op == 'X':
            print("Applied X")
            qml.X(wires=qubit)
        elif op == 'Z':
            print("Applied Z")
            qml.Z(wires=qubit)
        
    
    # Initialize measurement qubits for X-stabilizers with Hadamard
    qml.Hadamard(wires=9)
    qml.Hadamard(wires=10)
    qml.Hadamard(wires=11)
    qml.Hadamard(wires=12)
    
    # X-stabilizer 1 (measure qubit 9)
    qml.CNOT(wires=[9, 1])
    qml.CNOT(wires=[9, 2])
    
    # X-stabilizer 2 (measure qubit 10)
    qml.CNOT(wires=[10, 0])
    qml.CNOT(wires=[10, 1])
    qml.CNOT(wires=[10, 3])
    qml.CNOT(wires=[10, 4])
    
    # X-stabilizer 3 (measure qubit 11)
    qml.CNOT(wires=[11, 4])
    qml.CNOT(wires=[11, 5])
    qml.CNOT(wires=[11, 7])
    qml.CNOT(wires=[11, 8])
    
    # X-stabilizer 4 (measure qubit 12)
    qml.CNOT(wires=[12, 6])
    qml.CNOT(wires=[12, 7])
    
    # Final Hadamard on X-stabilizer measurement qubits
    qml.Hadamard(wires=9)
    qml.Hadamard(wires=10)
    qml.Hadamard(wires=11)
    qml.Hadamard(wires=12)
    
    # Z-stabilizer 1 (measure qubit 13)
    qml.CNOT(wires=[0, 13])
    qml.CNOT(wires=[3, 13])
    
    # Z-stabilizer 3 (measure qubit 14)
    qml.CNOT(wires=[1, 14])
    qml.CNOT(wires=[2, 14])
    qml.CNOT(wires=[4, 14])
    qml.CNOT(wires=[5, 14])
    
    # Z-stabilizer 2 (measure qubit 15)
    qml.CNOT(wires=[3, 15])
    qml.CNOT(wires=[4, 15])
    qml.CNOT(wires=[6, 15])
    qml.CNOT(wires=[7, 15])
    
    # Z-stabilizer 4 (measure qubit 16)
    qml.CNOT(wires=[5, 16])
    qml.CNOT(wires=[8, 16])


    
    # Measure stabilizer qubits in the Pauli-Z basis
    return [qml.expval(qml.PauliZ(wire)) for wire in range(9, 17)]
    



dev = qml.device('default.qubit', wires=17)

In [2]:
# Let's set parameters for our evaluation, right now we are chosing initial state |0> with X error on qubit 2

is_plus = False
i_want_errors = True
line_of_error = 2

In [3]:
# Noiseless simulation for actually doing one cycle of detection with the correction
surface_code_circuit = qml.QNode(surface_code_d3_circuit, dev)

# Run the circuit
results = surface_code_circuit(is_plus, i_want_errors, line_of_error)

# Convert tensor results of the expectation values to a list of floats
# Reordering of the value to : [X1,X2,X3,X4, Z1,Z2,Z3,Z4] !!!
exps = [round(float(result), 3) for result in results]
tempz3 = exps[5]
tempz2 = exps[6]
exps[5] = tempz2
exps[6] = tempz3
print("Measurement outcomes of stabilizer qubits as a list of expectation values:", exps)


Applied error X
Measurement outcomes of stabilizer qubits as a list of expectation values: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, 1.0]


# Little note:
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, 1.0]

 [X1,X2,X3,X4, Z1,Z2,Z3,Z4]

The |0> state in the computational (Z) basis can be written as a superposition in the X basis:
|0> = (|+> + |->)/sqrt2
The X basis states |+> and |-> are eigenstates of sigmax with eigenvalues +1 and -1 respectively.
To find the expectation value, we calculate 

The probability of measuring |+> is 1/2, giving contribution (+1)(1/2)
The probability of measuring |-> is 1/2, giving contribution (-1)(1/2)


Therefore:
<0|sigmax|0> = (+1)(1/2) + (-1)(1/2) = 0

Similar discussion applies when the initail state is |+> and we measure Z stabilizers.

1.0 is the eigenvalue of |0> in the Z basis
-1.0 is the eigenvalue of |1> in the Z basis

In [4]:
from pennylane import drawer

# A nice visualization to see we applied the right initial state and error
print(drawer.draw(surface_code_circuit)(is_plus, i_want_errors, line_of_error))

Applied error X
 0: ──────────╭X───────────────────────────────╭●──────────────────────────────────┤     
 1: ────╭X────│──╭X────────────────────────────│─────╭●────────────────────────────┤     
 2: ──X─│──╭X─│──│─────────────────────────────│─────│──╭●─────────────────────────┤     
 3: ────│──│──│──│──╭X─────────────────────────│──╭●─│──│────────╭●────────────────┤     
 4: ────│──│──│──│──│──╭X─╭X───────────────────│──│──│──│──╭●────│──╭●─────────────┤     
 5: ────│──│──│──│──│──│──│──╭X────────────────│──│──│──│──│──╭●─│──│────────╭●────┤     
 6: ────│──│──│──│──│──│──│──│────────╭X───────│──│──│──│──│──│──│──│──╭●────│─────┤     
 7: ────│──│──│──│──│──│──│──│──╭X────│──╭X────│──│──│──│──│──│──│──│──│──╭●─│─────┤     
 8: ────│──│──│──│──│──│──│──│──│──╭X─│──│─────│──│──│──│──│──│──│──│──│──│──│──╭●─┤     
 9: ──H─╰●─╰●─│──│──│──│──│──│──│──│──│──│───H─│──│──│──│──│──│──│──│──│──│──│──│──┤  <Z>
10: ──H───────╰●─╰●─╰●─╰●─│──│──│──│──│──│───H─│──│──│──│──│──│──│──│──│──│──│──│──┤

In [5]:
# We need to convert the expectation values into booleans in order to understand how to map errors and corrections
# It will make our lives easier, especially because we need to make a choice

def convert_stabilizer_measurements(measurements, is_plus_state=True):
    """
    Convert stabilizer measurements to binary error indicators.
    In the code it is possible to see that there is a threshold according to which an expctation value is converted 
    into an event.
    Usually the choice of an event happening is made when the probability is very high like 99.99...%
    In this case since it is an ideal case we choose 1e-10 as maximum error. 
    
    Args:
    measurements (list): List of 8 measurement results [X1,X2,X3,X4, Z1,Z2,Z3,Z4] !!!!
    is_plus_state (bool): Whether the initial state was |+> (True) or |0> (False)
    
    Returns:
    list: Binary list where 0 means "as expected" and 1 means "error detected"
    """
    n_stabilizers = len(measurements) // 2
    result = []
    
    for i, value in enumerate(measurements):
        if i < n_stabilizers:  
            # X stabilizers
            if is_plus_state:
                # For |+> state: X stabilizers should be +1
                result.append(0 if abs(value - 1.0) < 1e-10 else 1)
            else:
                # For |0> state: X stabilizers should be random (0.0 when averaged)
                result.append(0 if abs(value) < 0.1 else 1)
        else:  
            # Z stabilizers
            if is_plus_state:
                # For |+> state: Z stabilizers should be random (0.0 when averaged)
                result.append(0 if abs(value) < 0.1 else 1)
            else:
                # For |0> state: Z stabilizers should be +1
                result.append(0 if abs(value - 1.0) < 1e-10 else 1)
                
    return result

convert_stabilizer_measurements(exps, is_plus)

[0, 0, 0, 0, 0, 0, 1, 0]

## Due to extreme lack of time I implemented a lookup table based decoding
### This is not scalable and realistic, since we are doing only one cycle with a controlled error, the minimum weight perfect matching algorithm should be a better choice.


--Comment--

In practice, this syndrome degeneracy in the surface code is typically handled through several approaches:

Minimum Weight Perfect Matching (MWPM):


The most common approach when faced with degenerate syndromes, it chooses the correction with minimum weight. It would arbitrarily but consistently choose one of them. This works because both corrections are equivalent for preserving the logical state.


Repeated Measurements:


In practical implementations, stabilizers are measured repeatedly. This temporal dimension provides additional information. Can help distinguish between degenerate errors through their time evolution.

The key point is that for error correction to work, we don't need to identify the exact error location - we just need to apply a correction that returns the system to the code space and preserves the logical state. Any of the degenerate corrections will accomplish this.

In [6]:
import pennylane as qml
import numpy as np
"""
A distance-3 surface code can correct up to one physical error.

In general, a surface code of distance dd can correct up to floor(3−1 / 2)=1

This means a distance-3 surface code can detect and correct a single error, but it cannot reliably correct two or more errors.
"""


# Decode and return corrections
def lookup_table_decoder(syndrome, is_plus_state):
    
    # I removed the degenerate syndromes since we care about parity in the end.
    
    zero_state_lookup = {
        # For |0> state - detecting X errors through Z stabilizers
        # First 4 bits are ignored (random), last 4 are Z stabilizers
        # Single X errors
        (0,0,0,0, 1, 0, 0, 0): [(0, 'X')],    # X error on qubit 0
        (0,0,0,0, 0, 0, 1, 0): [(1, 'X')],    # X error on qubit 1
        # (0,0,0,0, 0, 0, 1, 0): [(2, 'X')],    # X error on qubit 2 degenerate
        (0,0,0,0, 1, 1, 0, 0): [(3, 'X')],    # X error on qubit 3
        (0,0,0,0, 0, 1, 1, 0): [(4, 'X')],    # X error on qubit 4
        (0,0,0,0, 0, 0, 1, 1): [(5, 'X')],    # X error on qubit 5
        (0,0,0,0, 0, 1, 0, 0): [(6, 'X')],    # X error on qubit 6
        # (0,0,0,0, 0, 1, 0, 0): [(7, 'X')],    # X error on qubit 7 degenerate
        (0,0,0,0, 0, 0, 0, 1): [(8, 'X')]     # X error on qubit 8

    }

    plus_state_lookup = {
        # For |+> state - detecting Z errors through X stabilizers
        # First 4 bits are X stabilizers, last 4 are ignored (random)
        # Single Z errors
        (0, 1, 0, 0, 0,0,0,0): [(0, 'Z')],    # Z error on qubit 0
        (1, 1, 0, 0, 0,0,0,0): [(1, 'Z')],    # Z error on qubit 1
        (1, 0, 0, 0, 0,0,0,0): [(2, 'Z')],    # Z error on qubit 2
        # (0, 1, 0, 0, 0,0,0,0): [(3, 'Z')],    # Z error on qubit 3 degenerate
        (0, 1, 1, 0, 0,0,0,0): [(4, 'Z')],    # Z error on qubit 4
        (0, 0, 1, 0, 0,0,0,0): [(5, 'Z')],    # Z error on qubit 5
        (0, 0, 0, 1, 0,0,0,0): [(6, 'Z')],    # Z error on qubit 6
        (0, 0, 1, 1, 0,0,0,0): [(7, 'Z')],    # Z error on qubit 7
        # (0, 0, 1, 0, 0,0,0,0): [(8, 'Z')],    # Z error on qubit 8 degenerate
    }


    # Return list of (qubit, op) pairs for correction
    syndrome_tuple = tuple(syndrome)
    if is_plus_state:
        correction = plus_state_lookup.get(syndrome_tuple, [])
    else:
        correction = zero_state_lookup.get(syndrome_tuple, [])
        
    return correction  



### Let's rerun the example but wit the corrections

In [7]:
# Create device and circuit with a noiseless simulation
dev = qml.device('default.qubit', wires=17)
surface_code_circuit = qml.QNode(surface_code_d3_circuit, dev)

# Run the circuit
results = surface_code_circuit(is_plus, i_want_errors, line_of_error)


# Convert tensor results of the expc values to a list of floats
# Reordering of the value to : [X1,X2,X3,X4, Z1,Z2,Z3,Z4]
exps = [round(float(result), 3) for result in results]
tempz3 = exps[5]
tempz2 = exps[6]
exps[5] = tempz2
exps[6] = tempz3
print("Measurement outcomes of stabilizer qubits as a list:", exps)
converted_synd = convert_stabilizer_measurements(exps, is_plus)
print("Converted outcomes of stabilizer qubits as a list:", converted_synd)

# Obtain the corrections from the lookup table
corrections = lookup_table_decoder(converted_synd, is_plus)
print("Corrections to be made : ", corrections)

Applied error X
Measurement outcomes of stabilizer qubits as a list: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, 1.0]
Converted outcomes of stabilizer qubits as a list: [0, 0, 0, 0, 0, 0, 1, 0]
Corrections to be made :  [(1, 'X')]


In [8]:
new_surface_code_circuit = qml.QNode(surface_code_d3_circuit, dev)
print("Generation of circuit.")

new_results = new_surface_code_circuit(is_plus, i_want_errors, line_of_error, corrections)
print("Results stored.")

Generation of circuit.
Applied error X
Applied X
Results stored.


In [9]:
print(drawer.draw(new_surface_code_circuit)(is_plus, i_want_errors, line_of_error, corrections))

Applied error X
Applied X
 0: ──────────╭X───────────────────────────────╭●──────────────────────────────────┤     
 1: ──X─╭X────│──╭X────────────────────────────│─────╭●────────────────────────────┤     
 2: ──X─│──╭X─│──│─────────────────────────────│─────│──╭●─────────────────────────┤     
 3: ────│──│──│──│──╭X─────────────────────────│──╭●─│──│────────╭●────────────────┤     
 4: ────│──│──│──│──│──╭X─╭X───────────────────│──│──│──│──╭●────│──╭●─────────────┤     
 5: ────│──│──│──│──│──│──│──╭X────────────────│──│──│──│──│──╭●─│──│────────╭●────┤     
 6: ────│──│──│──│──│──│──│──│────────╭X───────│──│──│──│──│──│──│──│──╭●────│─────┤     
 7: ────│──│──│──│──│──│──│──│──╭X────│──╭X────│──│──│──│──│──│──│──│──│──╭●─│─────┤     
 8: ────│──│──│──│──│──│──│──│──│──╭X─│──│─────│──│──│──│──│──│──│──│──│──│──│──╭●─┤     
 9: ──H─╰●─╰●─│──│──│──│──│──│──│──│──│──│───H─│──│──│──│──│──│──│──│──│──│──│──│──┤  <Z>
10: ──H───────╰●─╰●─╰●─╰●─│──│──│──│──│──│───H─│──│──│──│──│──│──│──│──│──

In [10]:
# Convert tensor results of the expc values to a list of floats
# Reordering of the value to : [X1,X2,X3,X4, Z1,Z2,Z3,Z4]
exps = [round(float(result), 3) for result in new_results]
tempz3 = exps[5]
tempz2 = exps[6]
exps[5] = tempz2
exps[6] = tempz3
print("Measurement outcomes of stabilizer qubits as a list:", exps)
convert_stabilizer_measurements(exps, is_plus)

Measurement outcomes of stabilizer qubits as a list: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]


[0, 0, 0, 0, 0, 0, 0, 0]

## The error has been corrected.